In [34]:
# ==============================
# Standard Library Imports
# ==============================
from enum import Enum
import random

# ==============================
# Third-party Library Imports
# ==============================
import matplotlib.pyplot as plt
from IPython.display import display, Latex
from matplotlib.ticker import MultipleLocator
import numpy as np  # Original numpy
import pennylane as qml
import scipy as sp

import torch

# Pennylane numpy
from pennylane import numpy as pnp 

In [35]:
# ==============================
# Setup for Quantum Computations
# ==============================

# PennyLane settings
dev = qml.device('default.mixed', wires=1)

# Define Hamiltonian for quantum computations
coeffs = [-0.5]
obs = [qml.PauliZ(0)]
hamiltonian = qml.Hamiltonian(coeffs, obs)

$$
Let, e^{-t/T_2} = e^{-\tau}
$$

$$
\frac{1}{2} 

\begin{bmatrix}

1 & e^{(i\phi - \tau)} \\
e^{(-i\phi - \tau)} & 1

\end{bmatrix}

=

\frac{1}{2} 

\begin{bmatrix}

1 & e^{i\phi} \sqrt{1 - \gamma} \\
e^{-i\phi} \sqrt{1 - \gamma} & 1

\end{bmatrix}$$

$$

Then,
\quad \gamma = 
1 - e^{-2 \tau}
$$

$$ 
e^{-\tau} = \sqrt{1 - \gamma}
$$

In [36]:
@qml.qnode(dev, interface='torch', diff_method='backprop')
def circuit(paras):
    global Phi_global
    phi = Phi_global
    
    # Set dephase factor 
    theta_x, phi_z = paras

    qml.RX(pnp.pi/2, wires = 0)
    # qml.Hadamard(wires=0)

    qml.ApproxTimeEvolution(hamiltonian, phi, 1)
    
    qml.RZ(phi_z, wires = 0)  # phi_z
    
    qml.RX(theta_x, wires = 0)  # theta_x
    
    return qml.density_matrix(wires = 0)

@qml.qnode(dev, interface='torch', diff_method='backprop')
def Post_selection_Dephase(phi):
    """ Take qnode from circuit_1 and calculate decoherence using kraus operator.
    
    Args:
        phi (float): Phi for Time-approximation. Pass by global variables:'Phi_global'

    Returns:
        qml.density_matrix: Density matrix of full qnode
    """
    
    global Paras_global, Phi_global, Gamma_ps
    Phi_global = phi
    
    # Get density matrix from circuit_1    
    density_matrix = torch.tensor(circuit(Paras_global), dtype=torch.complex128)
    qml.QubitDensityMatrix(density_matrix, wires = 0)
    
    # Kraus operator for 2*2 matrix
    K = torch.tensor([
        [pnp.sqrt(1 - Gamma_ps), 0],
        [0, 1]
    ], dtype=torch.complex128)
    
    Numerator = K @ density_matrix @ K.conj().T
    Denominator = torch.trace(Numerator)
    rho_ps = Numerator / Denominator

    qml.QubitDensityMatrix(rho_ps, wires = 0)
    # return qml.expval(hamiltonian) 
    return qml.density_matrix(wires = 0) 

In [41]:
@qml.qnode(dev, interface='torch', diff_method='backprop')
def circuit_test(phi):
    global Paras_global, Phi_global, Gamma_ps
    Phi_global = phi
    
    density_matrix = torch.tensor(circuit(Paras_global), dtype=torch.complex128)
    qml.QubitDensityMatrix(density_matrix, wires = 0)
    
    return qml.density_matrix(wires=0)

In [42]:
paras = torch.tensor([np.pi/2, np.pi/2], dtype=torch.complex128)
Paras_global = paras    # dtype=torch.complex128s

# float
phi = np.pi/2
Gamma_ps = 0.8
 
circuit_test(phi)

/var/folders/w1/ls_y8xbd21b7x0tbqwgfdcd00000gn/T/ipykernel_86754/3273077295.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  density_matrix = torch.tensor(circuit(Paras_global), dtype=torch.complex128)


tensor([[-3.9252e-17+0.0000e+00j,  5.5511e-17+1.1776e-16j],
        [ 5.5511e-17-1.6653e-16j,  1.0000e+00+0.0000e+00j]],
       dtype=torch.complex128)

In [ ]:
qml.qinfo.classical_fisher(circuit_test)(paras)

RuntimeError: shape '[2]' is invalid for input of size 4

In [ ]:
paras = torch.tensor([np.pi/2, np.pi/2], dtype=torch.complex128)
Paras_global = paras    # dtype=torch.complex128s

# float
phi = np.pi/2
Gamma_ps = 0.8
 
Post_selection_Dephase(phi)

/var/folders/w1/ls_y8xbd21b7x0tbqwgfdcd00000gn/T/ipykernel_86754/2676228542.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  density_matrix = torch.tensor(circuit(Paras_global), dtype=torch.complex128)


tensor([[-7.8505e-18+0.0000e+00j,  2.4825e-17+5.2663e-17j],
        [ 2.4825e-17-7.4476e-17j,  1.0000e+00+0.0000e+00j]],
       dtype=torch.complex128)

In [ ]:
qml.qinfo.classical_fisher(Post_selection_Dephase)(paras)

RuntimeError: shape '[2]' is invalid for input of size 4

In [ ]:
Phi_global = np.pi/2
paras = torch.tensor([np.pi/2, np.pi/2], dtype=torch.complex128)

paras

tensor([1.5708+0.j, 1.5708+0.j], dtype=torch.complex128)

In [ ]:
qml.qinfo.classical_fisher(circuit)(paras)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/autoray/autoray.py:1770: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:276.)
  return x.to(dtype=to_backend_dtype(dtype, like=x))


tensor([[ 3.5327e-16, -2.3551e-16],
        [-2.3551e-16,  1.5701e-16]], dtype=torch.float64,
       grad_fn=<MmBackward0>)

In [ ]:
def Cost_function(paras):
    """ Calculate Classical-Fisher-Information for qnode(=Post_selection_Dephase).
    
    Args:
        paras (Numpy array): [theta_init, tau_1, tau_2, tau_d1, tau_d2, tau_d3]

    Returns:
        _type_: CFI with minus(-) sign.
    """
    
    global Paras_global, Phi_global
    Paras_global = paras
    phi = pnp.array([Phi_global])
          
    CFI = qml.qinfo.classical_fisher(Post_selection_Dephase)(phi[0])
    
    return -CFI

In [ ]:
# paras = torch.zeros(2, dtype = torch.complex128)
# paras[:2] = np.pi/2

# paras

In [ ]:
paras = np.zeros(2)
paras[:2] = np.pi/2
Paras_global = paras

Phi_global = np.pi*2
phi = pnp.array([Phi_global], requires_grad = True)

CFI = qml.qinfo.classical_fisher(Post_selection_Dephase)(phi[0])

CFI


TypeError: The inputs given to jacobian must be either a Tensor or a tuple of Tensors but the given inputs has type <class 'pennylane.numpy.tensor.tensor'>.